# Spark SQL

diseñado para procesar datos estructurados y realizar consultas SQL en grandes volúmenes de datos. Es una parte fundamental del ecosistema de Spark que permite a los desarrolladores y científicos de datos trabajar con datos de manera eficiente y flexible.

## Fuentes de datos

- Archivos CSV
- Archivos JSON
- Archivos Parquet
- Archivos Avro
- Bases de datos SQL (como PostgreSQL, MySQL, SQL Server)
- Bases de datos NoSQL (como MongoDB, Cassandra)
- Apache Hive
- Apache HBase
- Archivos de texto plano
- Fuentes de datos en la nube (como Amazon S3, Google Cloud Storage, Azure Blob Storage)

## Funciones

### crear un contexto

In [ ]:
from pyspark.sql import SparkSession

# Crear una Spark session
spark = SparkSession.builder \
    .appName("EjemploPySparkCSV") \
    .getOrCreate()

### Conexión a un archivo csv

In [ ]:
ordenes_ruta = '../resources/ordenes.csv'
ordenes = spark.read.csv(ordenes_ruta, header=True, inferSchema=True)

### Crear una tabla temporal para hacer consultas sobre ella

In [ ]:
# Le qsuganmos nombres a las columnas
ordenes = ordenes.withColumnRenamed('', 'row') \
    .withColumnRenamed('0', 'id') \
    .withColumnRenamed('1', 'cliente_id') \
    .withColumnRenamed('2', 'valor') \
    .withColumnRenamed('3', 'fecha_compra') \
    .withColumnRenamed('4', 'status')

ordenes.createOrReplaceTempView("tabla_temporal")


### Realizar consutas

In [ ]:
resultado = spark.sql("SELECT * FROM tabla_temporal limit 3 ")
resultado.show()

## Otras funciones

### Agregar una nueva columna

In [ ]:
from pyspark.sql.functions import lit

ordenes = ordenes.withColumn('unos', lit(1))
ordenes.createOrReplaceTempView("tabla_temporal")
resultado = spark.sql("SELECT * FROM tabla_temporal where status <> 'Cancelada' limit 3 ")
resultado.show()

### Seleccionar columnas

In [ ]:
column = ordenes.select('valor', 'cliente_id')
column.show(3)

### Filtrar filas

In [ ]:
pedientes = ordenes.filter(ordenes['status'] == 'Pendiente')
pedientes.show(3)

### Group by

In [ ]:
ordenes.groupBy('status').sum('valor').show(3)


### Podemos hacer joins entre DataFrames

df = df.join(otro_df, df["columna_comun"] == otro_df["columna_comun"], "inner")

## Funciones de agregación

### Aplicar una ó mas funciones

**Nota:** funcitons literalmente me trae toas las funciones con las que puedo operar mis datos

In [ ]:
from pyspark.sql import functions as F
ordenes.groupBy('status').agg(
    F.sum('valor'), 
    F.avg('valor'), 
    F.min('valor'), 
    F.max('valor')).show()

### Aplicar una fuinción de agregación

In [ ]:
ordenes.groupBy('status').count().show()

## Funciones de fecha

### Definir fecha

In [ ]:
ordenes = ordenes.withColumn('fecha_compra_format', F.to_timestamp(ordenes['fecha_compra']))

### Componetes de la fecha

In [ ]:
ordenes = ordenes.withColumn('year', F.year('fecha_compra_format'))
ordenes = ordenes.withColumn('month', F.month('fecha_compra_format'))
ordenes = ordenes.withColumn('day', F.day('fecha_compra_format'))

In [ ]:
response = ordenes.select('day')
response.show(5)

### Operar con texto

In [ ]:
response = ordenes.withColumn('upper_name', F.lower('status'))
response.show(3)

## Funciones ventana

In [ ]:
from pyspark.sql.window import Window

window = Window.partitionBy('year').orderBy('id')

response = ordenes.withColumn('numero_columna', F.lag('valor',1).over(window))
response.show(20)

### Elimininar una columna

In [ ]:
response = response.drop('year', 'month', 'day', 'numero_columna')
response.show(1)

### Ver el esquema

In [ ]:
response.printSchema()

### Uso de WHERE

Podemos usar filter ó where de forma indistintiva en cualquier situación en la que queramos filtrar información  de neustra columnas.

In [ ]:
response.where("cliente_id = 2 and fecha_compra_format >= '2024-01-01'").show(5)